In [1]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import models
from comet_ml import Experiment
%matplotlib inline

In [2]:
experiment = Experiment(api_key="dZm2UV8sODS5eDYysEf8TzKNu", project_name="cart")

COMET WARNING: We detected that you are running inside a Ipython/Jupyter notebook environment but we cannot save your notebook source code. Please be sure to have installed comet_ml as a notebook server extension by running:
jupyter comet_ml enable
COMET INFO: 
COMET WARNING: Comet.ml support for Ipython Notebook is limited at the moment, automatic monitoring and stdout capturing is deactivated
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/syrios/cart/dfd1947180b34bf58bca3c814e2526c6



In [3]:
#env = gym.make("Pong-v0")
env = gym.make('CartPole-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


## Replay Memory

Store transitions that the agent observes so can be re-used later. By sampling from this randomly improves stability

In [4]:
Transition = namedtuple('Transition',
                       ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):
    
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0
        
    def push(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity
        
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

In [8]:
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 1.0
EPS_END = 0.001
EPS_DECAY = 10000
TARGET_UPDATE = 10
num_episodes = 3000
SAVE_EVERY = 100
model_save_name = "./models/cart_dqn_linear_policy.state"
start_from_trained = False
trained_model = None
N_ACTIONS = 2

experiment.log_parameter("batch size", BATCH_SIZE)
experiment.log_parameter("gamma", GAMMA)
experiment.log_parameter("eps start", EPS_START)
experiment.log_parameter("eps end", EPS_END)
experiment.log_parameter("eps decay", EPS_DECAY)
experiment.log_parameter("target update", TARGET_UPDATE)
experiment.log_parameter("num episodes", num_episodes)
experiment.log_parameter("start trained", start_from_trained)


# the policy network is used to play the game - aka actor
policy_net = models.NNPolicy().cuda()
if start_from_trained:
    policy_net.load_state_dict(torch.load(trained_model))

# the target net is used to predict Q values for next action
# we need 2 otherwise we would be using the same network
# in the actual and predicted values of our loss function
target_net = models.NNPolicy().cuda()
target_net.load_state_dict(policy_net.state_dict())
# sets training to false 
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)

steps_done = 0

def select_action(state):
    global steps_done
    # gen random number
    sample = random.random()
    # get threshold which decays from start to end 
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    print(eps_threshold)
    steps_done += 1
    # if exceed, pick best
    if sample > eps_threshold:
        # No gradients b/c not learning, just getting best one
        with torch.no_grad():
            return policy_net(state).max(1)[1].view(1,1)
    # else, random
    else:
        return torch.tensor([[random.randrange(2)]], device='cuda', 
                            dtype=torch.long)  

In [9]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return None
    
    transitions = memory.sample(BATCH_SIZE)
    # transpose the batch...
    batch = Transition(*zip(*transitions))
    # compute mask of transitions which didn't lead to ending game
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device="cuda", dtype=torch.uint8)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    
    # calculate the Q value of taking the state, action pairs which were taken
    # the gather basically takes the Q value for the action choosen
    # So if my input choose action 3 for the given state, that is what I would gather.
    # Basically, what is the Q value for what actually happened
    # Q value being the total expected value from taking an action given a state.
    # These are basically our predictions for learning
    state_action_values = policy_net(state_batch).gather(1, action_batch)
    
    next_state_values = torch.zeros(BATCH_SIZE).cuda()
    # get the best actions for the next states
    # detach is for speed so don't calc gradients
    target_net_results = target_net(non_final_next_states).max(1)[0].detach()
    next_state_values[non_final_mask] = target_net_results
    # the expected value of the Q(s,a) given from the policy net is the
    # reward given plus the discounted value of the Q value from taking the best
    # action at the next step
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    
    loss = F.smooth_l1_loss(state_action_values, 
                            expected_state_action_values.unsqueeze(1))
    
    # Optimize the policy net to become better and predicting Q values
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [10]:
running_reward = None
reward_sum = 0
for i_episode in range(num_episodes):
    state = env.reset()
    state = torch.from_numpy(state).float().unsqueeze(0).cuda()
    prev_state = None
    losses = []
    for t in range(1000):
        env.render()
        action = select_action(state)
        next_state, reward, done, _ = env.step(action.item())
        reward_sum += reward
        reward = torch.tensor([reward])
        reward = reward.cuda()
        
        if not done:
            next_state = torch.from_numpy(next_state).float().unsqueeze(0).cuda()
        else:
            next_state = None
        memory.push(state, action, next_state, reward)
        state = next_state
        optimize_model()
        if done:
            running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
            experiment.log_metric("reward sum", reward_sum, step=i_episode)
            reward_sum = 0
            experiment.log_metric("reward mean", running_reward, step=i_episode)
            break
    if i_episode % TARGET_UPDATE == 0:
        print("Episode: {}".format(i_episode))
        target_net.load_state_dict(policy_net.state_dict())
    if i_episode % SAVE_EVERY == 0:
        print("Saving Model...")
        torch.save(policy_net.state_dict(), model_save_name)

1.0
0.9999001049948335
0.9998002199786681
0.9997003449505049
0.9996004799093451
0.9995006248541901
0.9994007797840414
0.9993009446979004
0.999201119594769
0.9991013044736489
0.9990014993335417
0.9989017041734494
0.9988019189923744
0.9987021437893183
0.9986023785632838
0.9985026233132732
0.9984028780382888
Episode: 0
Saving Model...
0.998303142737333
0.9982034174094089
0.9981037020535187
0.9980039966686658
0.9979043012538527
0.9978046158080827
0.9977049403303587
0.9976052748196844
0.9975056192750626
0.9974059736954972
0.9973063380799915
0.997206712427549
0.9971070967371738
0.9970074910078697
0.9969078952386403
0.9968083094284899
0.9967087335764226
0.9966091676814427
0.9965096117425545
0.9964100657587623
0.996310529729071
0.9962110036524848
0.9961114875280087
0.9960119813546475
0.9959124851314061
0.9958129988572896
0.9957135225313031
0.9956140561524518
0.995514599719741
0.9954151532321763
0.9953157166887632
0.9952162900885071
0.995116873430414
0.9950174667134897
0.9949180699367399
0.9948

0.9571882220374915
0.9570926079960695
0.9569970035155736
0.9569014085950478
0.956805823233536
0.9567102474300825
0.9566146811837314
0.9565191244935272
0.9564235773585142
0.956328039777737
0.9562325117502403
0.9561369932750685
0.9560414843512669
0.9559459849778799
0.9558504951539529
0.9557550148785308
0.9556595441506589
0.9555640829693824
0.9554686313337468
0.9553731892427976
0.9552777566955802
0.9551823336911405
0.9550869202285239
0.9549915163067767
0.9548961219249446
0.9548007370820737
0.9547053617772102
0.9546099960094003
0.9545146397776905
0.9544192930811269
0.9543239559187564
Episode: 20
0.9542286282896255
0.9541333101927807
0.9540380016272691
0.9539427025921376
0.9538474130864331
0.9537521331092027
0.9536568626594938
0.9535616017363533
0.9534663503388289
0.9533711084659681
0.9532758761168183
0.9531806532904272
0.9530854399858428
0.9529902362021127
0.952895041938285
0.9527998571934078
0.952704681966529
0.9526095162566972
0.9525143600629605
0.9524192133843675
0.9523240762199665
0.95

0.9161196105535037
0.9160281031678938
0.915936604932565
0.9158451158466023
0.9157536359090908
0.9156621651191156
0.9155707034757621
0.9154792509781156
0.9153878076252616
0.9152963734162857
0.9152049483502737
0.9151135324263109
0.9150221256434836
0.9149307280008775
0.9148393394975788
0.9147479601326735
0.9146565899052477
0.9145652288143878
0.9144738768591802
0.9143825340387114
0.9142912003520681
0.9141998757983366
0.9141085603766039
0.9140172540859569
0.9139259569254825
0.9138346688942675
0.9137433899913994
0.9136521202159651
0.9135608595670519
0.9134696080437474
0.913378365645139
0.9132871323703142
0.9131959082183608
0.9131046931883665
0.9130134872794191
0.9129222904906066
0.912831102821017
0.9127399242697385
0.9126487548358592
0.9125575945184674
0.9124664433166516
0.9123753012295003
0.912284168256102
0.9121930443955453
0.912101929646919
0.9120108240093122
0.9119197274818136
0.9118286400635123
0.9117375617534973
0.911646492550858
0.9115554324546835
0.9114643814640635
0.9113733395780872

0.8770776938810494
0.8769900904919037
0.8769024958626589
0.8768149099924393
0.8767273328803685
0.8766397645255711
0.8765522049271716
0.8764646540842939
0.8763771119960628
0.876289578661603
0.8762020540800388
0.8761145382504951
0.8760270311720969
0.8759395328439691
0.8758520432652365
0.8757645624350244
0.875677090352458
0.8755896270166623
0.8755021724267631
0.8754147265818855
0.8753272894811552
0.8752398611236978
0.875152441508639
0.8750650306351048
0.8749776285022207
0.874890235109113
0.8748028504549076
0.8747154745387307
0.8746281073597086
0.8745407489169675
0.874453399209634
0.8743660582368346
0.8742787259976955
0.8741914024913439
0.8741040877169062
0.8740167816735094
0.8739294843602805
0.8738421957763464
0.8737549159208343
0.8736676447928714
0.8735803823915848
0.8734931287161022
0.8734058837655507
0.8733186475390583
0.8732314200357522
0.8731442012547603
0.8730569911952105
0.8729697898562305
0.8728825972369486
0.8727954133364926
0.8727082381539907
0.8726210716885712
0.872533913939362

0.8398691867591271
0.8397853040346573
0.8397014296980405
0.8396175637484381
0.8395337061850113
0.8394498570069217
0.8393660162133305
0.8392821838033995
0.8391983597762903
0.8391145441311649
0.8390307368671849
0.838946937983512
0.8388631474793088
0.838779365353737
0.8386955916059587
0.8386118262351365
0.8385280692404324
0.8384443206210092
0.8383605803760292
0.838276848504655
0.8381931250060491
0.8381094098793747
0.8380257031237942
0.8379420047384708
0.8378583147225677
0.8377746330752475
0.8376909597956735
0.8376072948830093
0.8375236383364182
0.8374399901550633
0.8373563503381083
0.8372727188847169
0.8371890957940527
0.8371054810652794
0.8370218746975608
0.836938276690061
0.8368546870419442
0.8367711057523741
0.8366875328205151
0.8366039682455315
0.8365204120265876
0.8364368641628476
0.8363533246534764
0.8362697934976384
0.8361862706944985
0.8361027562432212
0.8360192501429715
0.8359357523929143
0.8358522629922146
0.8357687819400376
0.8356853092355483
0.8356018448779122
0.83551838886629

...

0.003926346681460436
0.003926054061423536
0.003925761470647177
0.00392546890912843
0.0039251763768643755
0.003924883873852083
0.0039245914000886305
0.003924298955571091
0.003924006540296541
0.003923714154262058
0.003923421797464716
0.003923129469901589
0.0039228371715697595
0.003922544902466301
0.0039222526625882935
0.003921960451932809
0.003921668270496932
0.003921376118277737
0.003921083995272301
0.0039207919014777074
0.003920499836891033
0.003920207801509357
0.003919915795329756
0.003919623818349317
0.003919331870565115
0.003919039951974231
0.003918748062573746
0.0039184562023607435
0.003918164371332302
0.003917872569485505
0.003917580796817432
0.003917289053325168
0.003916997339005796
0.003916705653856396
0.00391641399787405
0.003916122371055848
0.003915830773398869
0.003915539204900195
0.003915247665556915
0.003914956155366113
0.003914664674324872
0.003914373222430275
0.003914081799679413
0.003913790406069369
0.00391349904159723
0.0039132077062600775
0.003912916400055005
0.0039126

0.003815823555961383
0.003815541987684436
0.003815260447562909
0.003814978935593984
0.0038146974517748507
0.003814415996102692
0.0038141345685746937
0.0038138531691880376
0.0038135717979399164
0.0038132904548275138
0.0038130091398480147
0.003812727852998605
0.0038124465942764763
0.0038121653636788134
0.0038118841612028043
0.0038116029868456346
0.003811321840604497
0.0038110407224765776
0.0038107596324590634
0.003810478570549148
0.003810197536744018
0.0038099165310408636
0.003809635553436872
0.0038093546039292383
0.003809073682515151
0.0038087927891918005
0.0038085119239563755
0.003808231086806072
0.003807950277738079
0.0038076694967495895
0.0038073887438377922
0.003807108018999885
0.003806827322233058
0.003806546653534504
0.0038062660129014143
0.003805985400330987
0.003805704815820414
0.003805424259366887
0.003805143730967604
0.0038048632306197593
0.0038045827583205465
0.0038043023140671593
0.0038040218978567975
0.0038037415096866547
0.0038034611495539267
0.003803180817455808
0.0038029

0.0037089328597450398
0.0037086619800032763
0.003708391127348135
0.003708120301776905
0.0037078495032868784
0.003707578731875344
0.003707307987539599
0.0037070372702769348
0.003706766580084643
0.0037064959169600146
0.0037062252809003477
0.003705954671902934
0.003705684089965064
0.0037054135350840377
0.003705143007257147
0.0037048725064816857
0.0037046020327549476
0.0037043315860742325
0.0037040611664368324
0.0037037907738400443
0.003703520408281162
0.003703250069757486
0.0037029797582663108
0.003702709473804933
0.003702439216370648
0.003702168985960757
0.003701898782572556
0.003701628606203343
0.0037013584568504135
0.003701088334511071
0.0037008182391826123
0.0037005481708623357
0.0037002781295475385
0.0037000081152355254
0.003699738127923593
0.003699468167609039
0.00369919823428917
0.0036989283279612833
0.0036986584486226794
0.003698388596270658
0.0036981187709025243
0.003697848972515579
0.003697579201107124
0.0036973094566744575
0.003697039739214888
0.003696770048725717
0.00369650038

0.0036058392146193006
0.003605578643726601
0.0036053180988896857
0.0036050575801059533
0.0036047970873727972
0.0036045366206876117
0.0036042761800477905
0.0036040157654507335
0.0036037553768938336
0.003603495014374485
0.00360323467789009
0.003602974367438041
0.0036027140830157357
0.003602453824620569
0.0036021935922499427
0.0036019333859012527
0.0036016732055718964
0.00360141305125927
0.0036011529229607766
0.003600892820673812
0.003600632744395776
0.0036003726941240648
0.0036001126698560833
0.0035998526715892284
0.0035995926993209
0.0035993327530484966
0.003599072832769423
0.0035988129384810774
0.003598553070180859
0.003598293227866174
0.003598033411534421
0.003597773621183002
0.0035975138568093176
0.0035972541184107736
0.0035969944059847706
0.003596734719528712
0.003596475059039998
0.0035962154245160376
0.003595955815954231
0.0035956962333519825
0.003595436676706694
0.0035951771460157747
0.0035949176412766273
0.0035946581624866558
0.003594398709643264
0.0035941392827438614
0.003593879

KeyboardInterrupt: 

In [ ]:
torch.save(policy_net.state_dict(), model_save_name)